# Thickness
* This Notebook is used to define the cum_thickness function.
* The cum_thickness functions runs throught the the interpreted well-data and outputs the following:
    * cumul_thick_list --> List containing cumulative thickness per geochemical unit. Used for appending to OG dataset.
    * cumul_thick_dict --> A dictionary containing (unit:total unit thickness) pairs.
-----------------------------------------------------------------------------------------------------------------   

# TODO:
* Maybe use some more generalization on the function by using regex expressions.
* Fix potentital bug for when the dataset  begins with volcaniclatic/siliciclastic facies.

## Importing Libraries and Data

In [1]:
# Importing libraries
import pandas as pd
import re

# Reading .csv
# data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';', decimal=',') ## Dataset1
data = pd.read_csv(r'Data/CSV/1-CS-002-PR.csv', sep=';', decimal=',') ##Dataset2
data.head()

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),3.65,NaN,NaN
1,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),1.12,NaN,NaN
2,1-CS-002-PR,Paranapanema,Siliciclastics,1.52,NaN,NaN
3,1-CS-002-PR,Paranapanema,Simple lava (basic | U. crust),11.60,37.28,0.650215
4,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),24.24,NaN,NaN


## Cumulative thickness of lava flows per geochem group
-----------------------------------------------------------------------------------------------------------------   
* This code runs throught the WellCad data and calculates the following:
    * Cumulative thickness of lavas per geochem group in a given well.
-----------------------------------------------------------------------------------------------------------------
* Outputs:
    * A cumul_thick_list containign the exact same number of values as OG dataframe. For appending.
    * A cumul_thick_dict containing (unit:cumulative thickness) pairs.

In [17]:
## Cumulative thickness function definition v0.1
## Seems correct.
def cum_thickness (data, cumul_thick, thick_dict):
    'Calculates the cumulative thickness of lava flows per geochemical group'
    'Input: data - pandas df derived from wellcad int; thick_dict - empty dict to append unit:cul_thick pair; cumul_thick - empty list to append data to' 
    'Output: a dictionary contaning (unit:cumulative thickness)' 
    'Output: a list with the same number of elements as input data containing cumulative flow thickness values. Used as a new row for input dataframe'
    
    # defining variables
    thick_counter = 0
    
    # Algorithm:
    for i in range(len(data)):
        if i == 0: # Setting up first iteration
            if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)' \
            or data['Facies'].iloc[i] == 'Simple lava (basic | rubbly flow top)': # If we're dealing with a new lava flow, grab thickness
                thick_counter += round(data['Flow thickness'].iloc[i])
                thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                cumul_thick.append(thick_counter)
            else:
                cumul_thick.append('0')
        else: # After first iteration
            if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before, keep going.
                if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)' \
                or data['Facies'].iloc[i] == 'Simple lava (basic | rubbly flow top)': # Look for facies with flow thickness data
                    thick_counter += round(data['Flow thickness'].iloc[i])
                    thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                    cumul_thick.append(thick_counter)
                else: # If no row with flow thickness data is being iterated
                    if re.search(r'\slava\s', data['Facies'].iloc[i]) != None: # If facies contain the word 'lava':
                        cumul_thick.append(thick_counter)
                    else:
                        cumul_thick.append('-')
            else: # If unit changes
                if data['Unit'].iloc[i] in thick_dict:  # If unit has already been added as a key to the thick_dict
                    thick_dict[data['Unit'].iloc[i-1]] = round(thick_counter)
                    thick_counter = round(thick_dict[data['Unit'].iloc[i]])
                    cumul_thick.append(thick_counter)
                else: # If a new unit is found
                    thick_dict[data['Unit'].iloc[i-1]] = round(thick_counter)
                    thick_counter = 0
                    if re.search(r'clastic', data['Facies'].iloc[i], re.IGNORECASE) != None:
                        cumul_thick.append('-')
                    else:
                        thick_counter = round(data['Flow thickness'].iloc[i])
                        thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                        cumul_thick.append(thick_counter)
                               
  

In [18]:
# Testing / Debugging
cs_pr_dict = {}
cs_pr_list = []

cum_thickness(data, cs_pr_list, cs_pr_dict)
print(cs_pr_dict)
print(cs_pr_list)
data['cumul thick'] = cs_pr_list
data.head(50)

{'Paranapanema': 37, 'Pitanga': 432, 'Esmeralda': 249, 'Vale do Sol': 135}
['0', 0, '-', 37, 37, 37, '-', 32, 32, 32, 55, 55, 55, 94, 94, 94, '-', 108, 114, 114, 114, 119, '-', 140, 140, 140, '-', 196, 196, 196, '-', 204, 269, 269, 269, '-', 303, 303, 303, 322, 322, 322, '-', 348, 348, 348, '-', 379, 379, 379, '-', 406, 406, 406, 432, 432, 432, 13, 26, 26, 26, '-', 75, 75, 75, '-', 87, 87, 87, '-', 158, 158, 158, 166, '-', 179, 179, 179, 195, 195, 195, 207, 207, 207, '-', 249, 249, 249, '-', 7, 14, 14, 14, 52, 52, 52, 63, 101, 101, 101, '-', 135, 135, 135]


,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion,cumul thick
0,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),3.65,NaN,NaN,0
1,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),1.12,NaN,NaN,0
2,1-CS-002-PR,Paranapanema,Siliciclastics,1.52,NaN,NaN,-
3,1-CS-002-PR,Paranapanema,Simple lava (basic | U. crust),11.60,37.28,0.650215,37
4,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),24.24,NaN,NaN,37
5,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),1.44,NaN,NaN,37
6,1-CS-002-PR,Pitanga,Siliciclastics,2.72,NaN,NaN,-
7,1-CS-002-PR,Pitanga,Simple lava (basic | U. crust),7.12,32.00,0.717500,32
8,1-CS-002-PR,Pitanga,Simple lava (basic | massive interior),22.96,NaN,NaN,32
9,1-CS-002-PR,Pitanga,Simple lava (basic | L. crust),1.92,NaN,NaN,32


-------------------------------------------------------------------------
# SEPARATION CELL
-------------------------------------------------------------------------

In [ ]:
# Regex debugging 
if re.search(r'\sU.\s', data['Facies'].iloc[0]) != None:
    print('Working!')
else:
    print('Not working :(')

In [ ]:
x = re.split(r'\s', data['Facies'].iloc[0])
print(re.search(r'U.', data['Facies'].iloc[0]))